In [ ]:
%matplotlib widget
from bmcs_cross_section.api import ReinfLayer, BarLayer, MKappa, TShape
mc = MKappa(high_kappa=0.00004)
mc.cs_design.cross_section_shape_.trait_set(H=500, B=300)
mc.cs_design.concrete.compression = 'EC2 with plateau'

In [ ]:
mc.cs_design.concrete.compression_.trait_set(
    f_c = 14.8, eps_cy=-0.0035, eps_cu=-0.006, E_cc=350000
)
mc.cs_design.concrete.tension_.trait_set(mu=0)
bl1 = BarLayer(name='steel 1', z=50, count=4, ds=20 )
bl1.matmod_.trait_set(f_sy = 435, f_st = 435)
mc.cross_section_layout.add_layer(bl1)
mc.interact()

In [ ]:
%%capture
import matplotlib.pylab as plt
from matplotlib import animation, rc
from IPython.display import HTML
import traits.api as tr
import bmcs_utils.api as bu
import numpy as np

fig = plt.figure(tight_layout=True)
fig.set_size_inches(10, 4)

In [ ]:
class CrossSectionAnimator(bu.Model):
    
    model = bu.Instance(bu.Model, mc)
    
    n_kappa = bu.Int(35)
    n_kappa_end = bu.Int(10)
    
    def init(self):
        self.kappa_T = np.hstack([
            np.linspace(0,self.model.high_kappa,self.n_kappa),
            np.ones((self.n_kappa_end,)) * self.model.high_kappa
        ])
    
    def plot(self, i):
        fig.clear()
        kappa = self.kappa_T[i]
        print('{},'.format(i), end='')
        #axes = self.model.subplots(fig)
        axes = fig.subplots(1,3)
        self.model.kappa_slider = kappa
        ax1, ax2, ax3 = axes
        ax22 = ax2.twiny()
        self.model.plot_strain_profile(ax1)
#        ax1.set_xlim(eps_min, eps_max)
        ax1.xaxis.set_ticks([-0.01, 0, 0.01])
        self.model.plot_mk_and_stress_profile(ax3, ax2, ax22)
        ax22.set_xlim(-20, 20)
        ax2.set_xlim(-500, 500)
        
    def mp4_video(self):
        n_t = len(self.kappa_T)
        # call the animator. blit=True means only re-draw the parts that have changed.
        anim = animation.FuncAnimation(fig, self.plot, init_func=self.init,
                                       frames=n_t, interval=300, blit=True)
        return anim.save("cross_section_animation.gif")
    
    def html5_video(self):
        n_t = len(self.kappa_T)
        # call the animator. blit=True means only re-draw the parts that have changed.
        anim = animation.FuncAnimation(fig, self.plot, # init_func=self.init,
                                       frames=n_t, interval=300, blit=True)
        return anim.to_html5_video()

In [ ]:
anim = CrossSectionAnimator(n_kappa=30, n_kappa_end=10)
anim.init()
with open('cross_section_animation.html','w') as html_video_file:
    html_video_file.write(anim.html5_video())

In [ ]:
html_video_file = open('cross_section_animation.html','r')
HTML(html_video_file.read())